## Web Scraping (Selenium) - Instagram - Descargar fotos de un hashtag

LIBRERIAS

In [15]:
#Librerias selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException 

import wget #Descargar archivos/imagenes
import time
import os #Manejar direcciones del path, creacion de carpeta

INICIALIZACION SELENIUM Y CHROME

In [16]:
#Iniciar Chrome con los parametros indicados
def iniciar_chrome():
   ruta = ChromeDriverManager(path='./chromedriver').install()
   options = Options()
   user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36"
   
   options.add_argument(f"user-agent={user_agent}")    
   options.add_argument("--start-maximized")
   options.add_argument("--disable-web-security") 
   options.add_argument("--disable-extensions") 
   options.add_argument("--disable-notifications") 
   options.add_argument("--no-first-run") 
   options.add_argument("--disable-blink-features=AutomationControlled")
   options.add_argument("--incognito")
   #Parametros a omitir al iniciar del ChromeDriver
   exp_opt = [
       'enable-automation', 
       'ignore-certification-errors', 
       'enable-logging' 
   ]
   options.add_experimental_option("excludeSwitches",exp_opt)

   #Parametros para definir preferencias en ChromeDriver
   prefs ={
       "profile.default_content_setting_values.notificacion" : 2, 
       "intl.accept_languages" : ["es_ES","es"], 
       "credentials_enable_service" : False 
   }   
   options.add_experimental_option("prefs",prefs)
   
   s = Service(ruta)
   driver = webdriver.Chrome(service=s, options=options)   
   return driver

FUNCIONES

In [17]:
def login_instagram(user_ig, pass_ig):
    driver.get("https://www.instagram.com/")
    #Busco textbox e ingreso usuario
    elem = wait.until(EC.visibility_of_element_located((By.NAME,"username")))
    elem.send_keys(user_ig)
    #Busco textbox e ingreso contrsaeña
    elem =wait.until(EC.visibility_of_element_located((By.NAME,"password")))
    elem.send_keys(pass_ig)
    #Boton Entrar
    elem =wait.until(EC.element_to_be_clickable((By.XPATH,"//div[contains(text(),'Entrar')]")))
    elem.click()    
    #Boton Guardar informacion
    elem =wait.until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Guardar')]")))
    elem.click()
    #Busco una publicacion para ver si el logueo fue correcto
    try:
        elem =wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,"article[role='presentation']")))
        print("Logueo correcto")
    except TimeoutException:
        print("Error de logueo")

In [22]:
def descargar_fotos_instagram(hashtag):
    driver.get(f"https://www.instagram.com/explore/tags/{hashtag}")
    url_fotos= set() #Declaro un conjunto vacio donde se guardan los url de las fotos, se usa set porque no permite repetidos

    while len(url_fotos) < 5: #defino un limite de fotos a descargar
        #Ejecutamos script java para desplazarnos al final del documento
        driver.execute_script("window,scrollTo(0, document.body.scrollHeight);")
        elementos = driver.find_elements(By.CSS_SELECTOR, "div._aagv") 
        #recorremos los cada imagen y guardamos su url       
        for elemento in elementos:
            try:
                url = elemento.find_element(By.CSS_SELECTOR, "img").get_attribute("src")                
                url_fotos.add(url)
            except:
                pass
    #Creamos una carpeta donde guardar las fotos
    if not os.path.exists(hashtag):
        os.mkdir(hashtag)
    for url_foto in url_fotos:
        wget.download(url_foto,hashtag)

             

########### MAIN ###########

In [ ]:
if __name__ == '__main__':
    
    driver = iniciar_chrome() #iniciamos selenium     
    wait = WebDriverWait(driver, 10) #Defino tiempo de espera maximo para cargar elementos
    
    #COLOCAR CREDENCIALES Y HASHTAG
    user_ig='????????'
    pass_ig='????????'
    hashtag='happycat'

    login_instagram(user_ig, pass_ig)
    descargar_fotos_instagram(hashtag)       
    driver.quit()